In [17]:
from pathlib import Path
base_dir = Path("/data1/ujiie/shinra/tohoku_bert")
for base_fn in base_dir.glob("**/**/tokens"):
    try:
        category = base_fn.parent.stem
        fn = base_fn.parent
        with open(fn / "attributes.txt", "r") as f:
            attributes = f.read()
        with open(base_dir / "attributes" / f"{category}.txt", "w") as f:
            f.write(attributes)
    except:
        print(base_fn.parent.stem, base_fn.parent.parent.stem)


Facility_Other Facility
Museum Facility
Car_Stop Facility
Sports_Facility Facility
Amusement_Park Facility
Archaeological_Place_Other Facility
Zoo Facility
Tunnel Facility
School Facility
Worship_Place Facility
Road_Facility Facility
Shopping_Complex Facility
Canal Facility
Castle Facility
Road Facility
Park Facility
Public_Institution Facility
Railway_Facility Facility
Medical_Institution Facility
Port Facility
Palace Facility
Research_Institute Facility
Station Facility
Theater Facility
Military_Base Facility
Cemetery Facility
Dam Facility
Power_Plant Facility
Railroad Facility
FOE_Other Facility
Bridge Facility
Line_Other Facility
Company JP-5
Compound JP-5
Person JP-5
Sports_Team Organization
Show_Organization Organization
Political_Party Organization
Organization_Other Organization
Political_Organization_Other Organization
Military Organization
Sports_Federation Organization
Sports_League Organization
Nonprofit_Organization Organization
International_Organization Organization


In [18]:
import json
annotations = {}
base_dir = Path("/data1/ujiie/shinra/tohoku_bert/")
for parent_dir in base_dir.glob("**/**/tokens"):
    category = parent_dir.parent.stem
    anns = {}
    with open(parent_dir.parent / f"{category}_dist.json", "r") as f:
        for line in f:
            line = line.rstrip()
            if not line:
                continue
            line = json.loads(line)
            if "text_offset" not in line:
                continue
            anns[(int(line["page_id"]), line["token_offset"]["start"]["line_id"], line["token_offset"]["start"]["offset"])] = line["attribute"]
        annotations[category] = anns


In [29]:
from collections import defaultdict
cdir = Path("/home/is/ujiie/shinra-pipeline/outputs")
attr_path = Path("/data1/ujiie/shinra/tohoku_bert/attributes")
output_dir = Path("/home/is/ujiie/shinra-pipeline/attributes")
# attr_list = ["Nonprofit_Organization", "Political_Party", "Show_Organization", "Sports_Federation", "Bridge", "Car_Stop", "Castle", "Company"]
attr_list = []
for fn in cdir.glob("*.jsonl"):
    category = fn.stem
    # if category != "Person":
    #     continue

    with open(attr_path / f"{category}.txt", "r") as f:
        attributes = [line for line in f.read().split("\n") if line != ""]
    
    original_attr = set(attributes)
    alignments = {}
    with open(fn, "r") as f:
        for line in f:
            line = line.rstrip()
            if not line:
                continue
            line = json.loads(line)
            if (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])) in annotations[category]:
                query = (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"]))
                if line["attribute"] in alignments:
                    alignments[line["attribute"]][annotations[category][query]] += 1
                else:
                    alignments[line["attribute"]] = defaultdict(int)
                    alignments[line["attribute"]][annotations[category][query]] += 1
            elif (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])-1) in annotations[category]:
                query = (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])-1)
                if line["attribute"] in alignments:
                    alignments[line["attribute"]][annotations[category][query]] += 1
                else:
                    alignments[line["attribute"]] = defaultdict(int)
                    alignments[line["attribute"]][annotations[category][query]] += 1
            elif (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])+1) in annotations[category]:
                query = (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])+1)
                if line["attribute"] in alignments:
                    alignments[line["attribute"]][annotations[category][query]] += 1
                else:
                    alignments[line["attribute"]] = defaultdict(int)
                    alignments[line["attribute"]][annotations[category][query]] += 1
            elif (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])+2) in annotations[category]:
                query = (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])+2)
                if line["attribute"] in alignments:
                    alignments[line["attribute"]][annotations[category][query]] += 1
                else:
                    alignments[line["attribute"]] = defaultdict(int)
                    alignments[line["attribute"]][annotations[category][query]] += 1
            elif (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])-2) in annotations[category]:
                query = (line['page_id'], int(line["token_offset"]["start"]["line_id"]), int(line["token_offset"]["start"]["offset"])-2)
                if line["attribute"] in alignments:
                    alignments[line["attribute"]][annotations[category][query]] += 1
                else:
                    alignments[line["attribute"]] = defaultdict(int)
                    alignments[line["attribute"]][annotations[category][query]] += 1

    new_attributes = []
    None_cnt = 0
    for attr in attributes:
        max_cat = None
        max_num = -1
        if attr in alignments:
            for key, value in alignments[attr].items():
                if category in attr_list:
                    print(category, attr, key, value)
                # print(attr, key, value)
                if max_num < value and key not in new_attributes:
                    max_cat = key
                    max_num = value
            if max_cat is not None:
                new_attributes.append(max_cat)
                # print(new_attributes)
            else:
                new_attributes.append(f"None{None_cnt}")
                None_cnt += 1
        else:
            new_attributes.append(f"None{None_cnt}")
            None_cnt += 1
            print(category)
    new_attributes_set = set(new_attributes)
    assert len(new_attributes) == len(new_attributes_set)
    with open(output_dir / f"{category}.txt", "w") as f:
        f.write("\n".join(new_attributes))
        remain = original_attr - new_attributes_set
        f.write("\n" + ",".join(list(remain)))

        # f.write("\n\n" + "\n".join(attributes))
        

Park
Archaeological_Place_Other
Bridge
Bridge
Power_Plant
Power_Plant
Power_Plant
Canal
Canal
Canal
Public_Institution
Show_Organization
Show_Organization
Car_Stop
Castle
Castle
Castle
Castle
Castle
Castle
Castle
Castle
Castle
Castle
Castle
Sports_Federation
Sports_Federation
Cemetery
Cemetery
Cemetery
Cemetery
Cemetery
Cemetery
Cemetery
Cemetery
Research_Institute
Dam
Dam
Dam
Sports_Team
Sports_Team
Sports_Team
Road
Road
Road
FOE_Other
FOE_Other
FOE_Other
Road_Facility
School
School
School
School
Facility_Other
Line_Other
Line_Other
Line_Other
Line_Other
Line_Other
Shopping_Complex
Medical_Institution
Station
Worship_Place
Worship_Place
